<a href="https://colab.research.google.com/github/NormLorenz/ai-llm-openai-mcp/blob/main/openai-mcp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using OpenAI Agents with a MCP Server

In [1]:
# Install required packages

!pip install --upgrade pip
!pip install fastmcp openai nest_asyncio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.9 MB/s  0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 2.0.1
    Uninstalling wrapt-2.0.1:
      Successfully uninstalled wrapt-2.0.1
  Attempting uninstall: referencing
    Found existing installation: referencing 0.37.0
    Uninstalling referencing-0.37.0:
      Successfully uninstalled referencing-0.37.0
  Attempting uninstall: opentelemetry-api
    Found existing installation: opentelemetry-api 1.37.0
    Uninstalling opentelemetry-api-1.37.0:
      Successfully uninstalled opentelemetry-api-1.37.0
  Attempting uninstall: opentelemetry-semantic-conventions
    Found existing installation: opentelemetry-semantic-conventions 0.58b0
    Uninstalling opentelemetry-semantic-convention

This code sets up and runs a FastMCP server that acts as a weather service.

It uses fastmcp to create an agent that can interact with the National Weather Service (NWS) API. It defines several Tools like get_forecast, get_alerts, and health_check which perform specific actions. The server is then launched in a background thread to make these tools and resources accessible via HTTP, specifically configured for notebook compatibility.

The MCP server is based upon code found at https://github.com/lxchst/weather-server-python/blob/main/src/weather/server.py

In [2]:
# The MCP Server

from fastmcp import FastMCP
import nest_asyncio
import threading
import time
from typing import Any
import httpx

# Constants
NWS_API_BASE = "https://api.weather.gov"
USER_AGENT = "weather-app/1.0"

nest_asyncio.apply()

mcp = FastMCP(
    name="WeatherServer",
    instructions="Provides an up to date weather forecast and alerts for any location and also includes a health check."
)

async def make_nws_request(url: str) -> dict[str, Any] | None:
    """Make a request to the NWS API with proper error handling."""
    headers = {"User-Agent": USER_AGENT, "Accept": "application/geo+json"}
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(url, headers=headers, timeout=30.0)
            response.raise_for_status()
            return response.json()
        except Exception:
            return None


def format_alert(feature: dict) -> str:
    """Format an alert feature into a readable string."""
    props = feature["properties"]
    return f"""
        Event: {props.get("event", "Unknown")}
        Area: {props.get("areaDesc", "Unknown")}
        Severity: {props.get("severity", "Unknown")}
        Description: {props.get("description", "No description available")}
        Instructions: {props.get("instruction", "No specific instructions provided")}
        """

@mcp.tool
def greet(name: str) -> str:
    """Greet a person by their name.

    Args:
        name: The name of the person to greet.
    """
    return f"Hello, {name}!"


@mcp.tool()
async def get_alerts(state: str) -> str:
    """Get weather alerts for a US state.

    Args:
        state: Two-letter US state code (e.g. CA, NY)
    """
    url = f"{NWS_API_BASE}/alerts/active/area/{state}"
    data = await make_nws_request(url)

    if not data or "features" not in data:
        return "Unable to fetch alerts or no alerts found."

    if not data["features"]:
        return "No active alerts for this state."

    alerts = [format_alert(feature) for feature in data["features"]]
    return "\n---\n".join(alerts)


@mcp.tool()
async def get_forecast(latitude: float, longitude: float) -> str:
    """Get weather forecast for a location.

    Args:
        latitude: Latitude of the location
        longitude: Longitude of the location
    """
    # First get the forecast grid endpoint
    points_url = f"{NWS_API_BASE}/points/{latitude},{longitude}"
    points_data = await make_nws_request(points_url)

    if not points_data:
        return "Unable to fetch forecast data for this location."

    # Get the forecast URL from the points response
    forecast_url = points_data["properties"]["forecast"]
    forecast_data = await make_nws_request(forecast_url)

    if not forecast_data:
        return "Unable to fetch detailed forecast."

    # Format the periods into a readable forecast
    periods = forecast_data["properties"]["periods"]
    forecasts = []
    for period in periods[:5]:  # Only show next 5 periods
        forecast = f"""
        {period["name"]}:
        Temperature: {period["temperature"]}°{period["temperatureUnit"]}
        Wind: {period["windSpeed"]} {period["windDirection"]}
        Forecast: {period["detailedForecast"]}
        """
        forecasts.append(forecast)

    return "\n---\n".join(forecasts)


@mcp.tool()
def health_check():
    """Returns the health status of the server."""
    return {"status": "ok"}


# Define the function to run the server
def run_server():
    # Use transport="streamable-http" for compatibility with notebooks/Colab
    print("🚀 Starting FastMCP server in background thread...")
    mcp.run(transport="streamable-http", host="0.0.0.0", port=8000)

# Start the server in a separate thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Give the server a moment to start up
time.sleep(5)
print("✅ Server should be running. Access it at http://localhost:8000/mcp")


🚀 Starting FastMCP server in background thread...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


╭──────────────────────────────────────────────────────────────────────────────╮                  
                 │                                                                              │                  
                 │                         ▄▀▀ ▄▀█ █▀▀ ▀█▀ █▀▄▀█ █▀▀ █▀█                        │                  
                 │                         █▀  █▀█ ▄▄█  █  █ ▀ █ █▄▄ █▀▀                        │                  
                 │                                                                              │                  
                 │                                FastMCP 2.14.1                                │                  
                 │                                                                              │                  
                 │                                                                              │                  
                 │                   🖥  Server name: WeatherServer                              │                  
                 │                                                                              │                  
                 │                   📦 Transport:   HTTP                                       │                  
                 │                   🔗 Server URL:  http://0.0.0.0:8000/mcp                    │                  
                 │                                                                              │                  
                 │                   📚 Docs:        https://gofastmcp.com                      │                  
                 │                   🚀 Hosting:     https://fastmcp.cloud                      │                  
                 │                                                                              │                  
                 ╰──────────────────────────────────────────────────────────────────────────────╯

[12/22/25 19:42:14] INFO     Starting MCP server 'WeatherServer' with transport 'streamable-http' on ]8;id=877892;file:///usr/local/lib/python3.12/dist-packages/fastmcp/server/server.py\server.py]8;;\:]8;id=292549;file:///usr/local/lib/python3.12/dist-packages/fastmcp/server/server.py#2618\2618]8;;\
                             http://0.0.0.0:8000/mcp                                                               

INFO:     Started server process [153]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


✅ Server should be running. Access it at http://localhost:8000/mcp


This code connects to a local FastMCP server to retrieve and then call its available tools. It defines several asynchronous functions, each designed to interact with a specific tool on the server (like greet, get_alerts, get_forecast, health_check, and list_tools), and then it executes these functions to demonstrate how to use the server's capabilities.

In [3]:
# First try to connect to the MCP server.

import asyncio
from fastmcp import Client

client = Client("http://localhost:8000/mcp")

async def call_greet(name: str):
    async with client:
        result = await client.call_tool("greet", {"name": name})
        print(result)


async def call_get_alerts(name: str):
    async with client:
        result = await client.call_tool("get_alerts", {"state": name})
        print(result)


async def call_get_forecast(latitude: float, longitude: float):
    async with client:
        result = await client.call_tool("get_forecast", {"latitude": latitude, "longitude": longitude})
        print(result)


async def call_health_check():
    async with client:
        result = await client.call_tool("health_check")
        print(result)


async def call_list_tools():
    async with client:
        result = await client.list_tools()
        for tool in result:
            print(tool)

asyncio.run(call_greet("Norm"))
asyncio.run(call_get_alerts("WA"))
asyncio.run(call_health_check())
asyncio.run(call_get_forecast(47.7179, -116.9516))
asyncio.run(call_list_tools())


INFO:     127.0.0.1:35584 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:35592 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:35602 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:35612 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:35628 - "POST /mcp HTTP/1.1" 200 OK
CallToolResult(content=[TextContent(type='text', text='Hello, Norm!', annotations=None, meta=None)], structured_content={'result': 'Hello, Norm!'}, meta=None, data='Hello, Norm!', is_error=False)
INFO:     127.0.0.1:35636 - "DELETE /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:35652 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:35666 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:35680 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:35696 - "POST /mcp HTTP/1.1" 200 OK


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

INFO:     127.0.0.1:35704 - "POST /mcp HTTP/1.1" 200 OK
CallToolResult(content=[TextContent(type='text', text="\n        Event: Wind Advisory\n        Area: San Juan County\n        Severity: Moderate\n        Description: * WHAT...Southeast winds 20 to 30 mph with gusts up to 45 mph.\n\n* WHERE...Island County, Lowlands of Western Skagit and Northwestern\nSnohomish Counties, Lowlands of Western Whatcom County, and San\nJuan County.\n\n* WHEN...Until 7 PM PST this evening.\n\n* IMPACTS...Gusty winds could blow around unsecured objects. Tree\nlimbs could be blown down and a few power outages may result.\n        Instructions: Use extra caution when driving, especially if operating a high\nprofile vehicles. Secure outdoor objects.\n        \n---\n\n        Event: Flood Warning\n        Area: Mason, WA\n        Severity: Severe\n        Description: ...The Flood Warning is extended for the following rivers in\nWashington...\n\nSkokomish River At Potlatch affecting Mason County.\n\n* WHAT.

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

INFO:     127.0.0.1:35758 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:35774 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:35778 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:35792 - "POST /mcp HTTP/1.1" 200 OK


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


INFO:     127.0.0.1:35804 - "POST /mcp HTTP/1.1" 200 OK
CallToolResult(content=[TextContent(type='text', text='\n        Today:\n        Temperature: 41°F\n        Wind: 3 to 7 mph S\n        Forecast: A slight chance of rain after 4pm. Partly sunny. High near 41, with temperatures falling to around 38 in the afternoon. South wind 3 to 7 mph. Chance of precipitation is 20%.\n        \n---\n\n        Tonight:\n        Temperature: 33°F\n        Wind: 6 mph SE\n        Forecast: A slight chance of rain before 10pm. Mostly cloudy, with a low around 33. Southeast wind around 6 mph. Chance of precipitation is 20%.\n        \n---\n\n        Tuesday:\n        Temperature: 42°F\n        Wind: 5 mph SE\n        Forecast: Rain likely after 4pm. Mostly cloudy. High near 42, with temperatures falling to around 39 in the afternoon. Southeast wind around 5 mph. Chance of precipitation is 70%. New rainfall amounts less than a tenth of an inch possible.\n        \n---\n\n        Tuesday Night:\n      

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

This code demonstrates how to integrate a FastMCP server with an OpenAI agent. It first connects to the local FastMCP server to retrieve the tools it exposes (like weather forecast and alerts). These tools are then used to create an OpenAI agent. Finally, the agent is run with a user input, and it leverages the provided FastMCP tools to find the answer, in this case, asking about the weather in Seattle.

In [ ]:
# OpenAI Agents code

from fastmcp import Client
import openai
import asyncio
import os
from google.colab import userdata

# Print the key prefixes to help with any debugging
openai_api_key = userdata.get("OPENAI_API_KEY")

# Initialize the OPENAI_API_KEY as an environment variable
os.environ["OPENAI_API_KEY"] = openai_api_key

# Assuming MCP_SERVER_URL is defined from previous cells
MCP_SERVER_URL = "http://localhost:8000/mcp"

agent_id = 0

async def main():
    # Connect to the MCP server
    mcp_client = Client(MCP_SERVER_URL)

    # Load tools from the server
    tools = await mcp_client.list_tools()

     # Initialize OpenAI client (ensure OPENAI_API_KEY is set as an environment variable)
    openai_client = openai.OpenAI()

    agent = openai_client.agents.create(
        model="gpt-4.1",
        name="Integrated-MCP-Agent",
        instructions="You can call MCP tools when needed.",
        tools=tools # <-- MCP tools injected here
    )

    # Ask the agent something that uses the MCP tool
    response = agent.run(
        agent_id=agent.id,
        input="What is the weather in Seattle?"
    )

    print(response.output_text)

asyncio.run(main())


In [ ]:
from fastmcp import Client
import openai
import asyncio
import os
from google.colab import userdata

# Initialize API key
openai_api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key

MCP_SERVER_URL = "http://localhost:8000/mcp"

async def main():
    """Main entry point"""
    # Connect to the MCP server
    mcp_client = Client(MCP_SERVER_URL)

    # Load tools from the server
    mcp_tools = await mcp_client.list_tools()

    # Convert MCP tools to OpenAI format (you'll need to implement this)
    openai_tools = convert_mcp_to_openai_format(mcp_tools)

    # Initialize OpenAI client
    openai_client = openai.OpenAI()

    # Create the agent
    openai_client = OpenAI()
    agent = openai_client.agents.create(
        model="gpt-4.1",
        name="MCP Agent",
        instructions="You can call MCP tools when needed.",
        tools=agent_tools # <-- MCP tools injected here
    )

    # Create a thread
    thread = openai_client.beta.threads.create()

    # Add a message to the thread
    message = openai_client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content="What is the weather in Seattle?"
    )

    # Create and poll the run
    run = openai_client.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=assistant.id
    )

    # Handle tool calls if needed
    if run.status == 'requires_action':
        # Process tool calls with your MCP server
        # Submit tool outputs back to OpenAI
        pass

    # Get the response
    if run.status == 'completed':
        messages = openai_client.beta.threads.messages.list(
            thread_id=thread.id
        )
        print(messages.data[0].content[0].text.value)

def convert_mcp_to_openai_format(mcp_tools):
    """Convert fastMCP tools to OpenAI Agent tool format"""
    agent_tools = []
    for tool in mcp_tools:
        agent_tools.append({
            "type": "function",
            "name": tool["name"],
            "description": tool.get("description", ""),
            "parameters": tool["input_schema"]  # <-- key conversion
        })
    return agent_tools

asyncio.run(main())

In [24]:
from fastmcp import Client
from openai import OpenAI
import asyncio
import os
from google.colab import userdata

# Initialize API key
openai_api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key

MCP_SERVER_URL = "http://localhost:8000/mcp"

async def main():
    # Connect to your MCP server
    mcp_client = Client(MCP_SERVER_URL)

    # Fetch FastMCP tool definitions within the context manager
    mcp_tools = []
    async with mcp_client:
        mcp_tools = await mcp_client.list_tools()

    # Convert to OpenAI Assistant tool format
    agent_tools = []
    for tool in mcp_tools:
        # OpenAI function tools require details to be nested under a 'function' key
        agent_tools.append({
            "type": "function",
            "function": {
                "name": tool.name,
                "description": tool.description if tool.description else "",
                "parameters": tool.inputSchema
            }
        })

    # Create the assistant (formerly 'agent')
    openai_client = OpenAI()
    assistant = openai_client.beta.assistants.create(
        model="gpt-4.1",
        name="MCP Assistant",
        instructions="Use the provided tools when helpful.",
        tools=agent_tools
    )

    print("Assistant created:", assistant.id)

asyncio.run(main())

INFO:     127.0.0.1:58660 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:58672 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:58674 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:58680 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:58692 - "DELETE /mcp HTTP/1.1" 200 OK


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Assistant created: asst_yTb7jwnwhPXlt8PGzptcnR7x
